In [1]:
from elasticsearch import Elasticsearch, helpers
from datetime import datetime, timezone
import uuid
from dateutil import parser
import json
import time

In [2]:
# create python client for elasticsearch
es = Elasticsearch("http://localhost:9200")

In [3]:
index_name ="tweets"

In [25]:
if es.indices.exists(index_name):
     es.indices.delete(index=index_name)

/var/folders/m1/fg0zsph94pj5b7vblgt01b580000gn/T/ipykernel_5663/4044851082.py:1: DeprecationWarning: Using positional arguments for APIs is deprecated and will be disabled in 8.0.0. Instead use only keyword arguments for all APIs. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  if es.indices.exists(index_name):


In [5]:
#  mapping the index
body = {
  "mappings": {
        "properties": {
          "id":{
              "type": "keyword",
              "ignore_above": 256
              },

          "text": {
              "type" : "text",
              "analyzer": "standard",    
              },

          "date": {
              "type": "date",
              },

          "location":{
              "type" : "geo_point"
              },

    
              } }}

In [6]:
#  query return the docs contain word "flood" at specific range of time at boundary box coordenates
query_body =  {"query":{
      "bool": {
        "must": [
          {
            "match" : {
              "text":{
              "query":"flood" 
          }}},
          {
          "range": {
          "date": {
          "time_zone": "+00:00",  
          "gte": "2013-01-01T22:10:00", 
          "lte": "2013-12-20T22:12:00"                  
      }}},
        
        {
          "geo_bounding_box": {
            "location": {
              
              "top_left": {
                "lat": 90.000,
                "lon": -180.000
              },
              "bottom_right": {
                "lat": -90.000,
                "lon": 180.000
              }
            }
          }
        }   
        ]
      }}}

In [26]:
#  create index with "tweet name"
es.indices.create(index=index_name, ignore=400, body=body)

/var/folders/m1/fg0zsph94pj5b7vblgt01b580000gn/T/ipykernel_5663/866937466.py:2: DeprecationWarning: The 'body' parameter is deprecated for the 'create' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.indices.create(index=index_name, ignore=400, body=body)


{'acknowledged': True, 'shards_acknowledged': True, 'index': 'tweets'}

In [27]:
#  read the json file of tweets and insert it to the tweet index
def generator():
    
    index_name = 'tweets'
    i=0
    for line in open('../data/boulder_flood_geolocated_tweets-1.json', 'r' ,encoding='utf-8'):
           
            data_ = json.loads(line)

            # convert time with "yyyy-mm-ddThh:mm:ss" without the time zone
            tweet_date = parser.parse(data_["created_at"])
            data_["created_at"] = tweet_date.replace(tzinfo=None)
            
            doc = { 
                 '_index': index_name,
                    "id" : int(data_["id"]),
                    "text" : data_["text"],
                    "date" : data_["created_at"] ,
                    "location" : data_["coordinates"],
                            }
            i+=1
            if i% 4000 == 0:
                 time.sleep(2)

            yield doc
            

            
                
         
       

In [28]:
helpers.bulk(es, generator(), chunk_size=4000, raise_on_error=True ,raise_on_exception=True, yield_ok=True)

(18821, [])

In [29]:
# search query 
result = es.search(
      index = index_name,
      body = query_body
  )

/var/folders/m1/fg0zsph94pj5b7vblgt01b580000gn/T/ipykernel_5663/1477574662.py:2: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  result = es.search(


In [30]:
# documents resulted from query
result['hits']

{'total': {'value': 194, 'relation': 'eq'},
 'max_score': 6.3621383,
 'hits': [{'_index': 'tweets',
   '_id': '0G5yJoUBL-qfb7xN22D0',
   '_score': 6.3621383,
   '_source': {'id': 378611244619415552,
    'text': 'Flood? What flood? #Denver #cowx http://t.co/TgKowYe5lr',
    'date': '2013-09-13T20:08:49',
    'location': {'type': 'Point', 'coordinates': [-104.989221, 39.743126]}}},
  {'_index': 'tweets',
   '_id': '425yJoUBL-qfb7xN3mwo',
   '_score': 6.1421237,
   '_source': {'id': 378292312524455936,
    'text': 'We\'ve finally admitted it is the 100yr flood #noshit “@dailycamera: Flood expert: Boulder experiencing a 100-year flood. #boulderflood"',
    'date': '2013-09-12T23:01:30',
    'location': {'type': 'Point',
     'coordinates': [-89.99813896, 35.14676299]}}},
  {'_index': 'tweets',
   '_id': '_m5yJoUBL-qfb7xN3nQr',
   '_score': 5.910203,
   '_source': {'id': 378068649430380544,
    'text': "Hello 2am. I'm up making sure our house doesn't flood. #boulderflood #fortcollins #larim